Test our "current" selection.

This uses sample activities that we expect to be current or not. These are taken from PWYFs test of their own function, so should be a good test. Our code behaves as expected.

Issue: https://github.com/OpenDataServices/iati-notebooks/issues/4#issuecomment-565536737


In [0]:
# Copy/pasted from https://colab.research.google.com/drive/1ozJ_Soc_Hj6Qi3H2lszcmRfz763TZ3lj#scrollTo=tRGl8fkYpyN2
# but with fake NOW

import datetime as dt
from dateutil.relativedelta import relativedelta

NOW = dt.datetime(2010, 1, 1)

def current_dict(activity):
  status_check = False
  planned_end_date_check = False
  actual_end_date_check = False
  transaction_date_check = False

  # print("Activity {} of {}".format(count, len(big_iati)))
  
  if activity.xpath("activity-status[@code=2]"):
    status_check = True

  if activity.xpath("activity-date[@type=3]/@iso-date"):
    date_time_obj = dt.datetime.strptime(activity.xpath("activity-date[@type=3]/@iso-date")[0], '%Y-%m-%d')
    if date_time_obj > (NOW - relativedelta(years=1)):
      planned_end_date_check = True
  
  if activity.xpath("activity-date[@type=4]/@iso-date"):
    date_time_obj = dt.datetime.strptime(activity.xpath("activity-date[@type=4]/@iso-date")[0], '%Y-%m-%d')
    if date_time_obj > (NOW - relativedelta(years=1)):
      actual_end_date_check = True

  if activity.xpath("transaction/transaction-type[@code=2 or @code=3 or @code=4]"):
    dates = activity.xpath("transaction[transaction-type[@code=2 or @code=3 or @code=4]]/transaction-date/@iso-date")
    date_truths = [dt.datetime.strptime(date, '%Y-%m-%d') > (NOW - relativedelta(years=1)) for date in dates]
    if True in date_truths:
      transaction_date_check = True

  pwyf_current = status_check or planned_end_date_check or actual_end_date_check or transaction_date_check

  return {
    'iati-id': activity.findtext('iati-identifier'),
    'status_check': status_check, 
    'planned_end_date_check': planned_end_date_check, 
    'actual_end_date_check': actual_end_date_check, 
    'transaction_date_check': transaction_date_check,
    'pwyf_current': pwyf_current,
  }
  

In [0]:
# Derived from https://github.com/pwyf/latest-index-indicator-definitions/blob/6a0f652e79a806b5cabcfe8c33a9477dd9605e29/test_definitions/step_definitions.py

from lxml import etree

xml = '''
<iati-activity>
  <activity-status code="2"/>
</iati-activity>
'''

activity = etree.fromstring(xml)
result = current_dict(activity)['pwyf_current']

assert result is True


xml = '''
<iati-activity>
  <activity-status code="3"/>
</iati-activity>
'''

activity = etree.fromstring(xml)
result = current_dict(activity)['pwyf_current']

assert result is False


xml = '''
<iati-activity>
  <activity-status code="3"/>
  <activity-date type="4" iso-date="2009-09-01" />
</iati-activity>
'''

activity = etree.fromstring(xml)
result = current_dict(activity)['pwyf_current']

assert result is True


xml = '''
<iati-activity>
  <activity-status code="3"/>
  <activity-date type="3" iso-date="2011-01-01" />
</iati-activity>
'''

activity = etree.fromstring(xml)
result = current_dict(activity)['pwyf_current']

assert result is True


xml = '''
<iati-activity>
  <activity-status code="3"/>
  <transaction>
    <transaction-type code="2" />
    <transaction-date iso-date="2009-12-01" />
  </transaction>
</iati-activity>
'''

activity = etree.fromstring(xml)
result = current_dict(activity)['pwyf_current']

assert result is True